In [1]:
!pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks

In [15]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key=os.getenv("PINECONE_API_KEY")

In [4]:
from langchain_community.retrievers import PineconeHybridSearchRetriever


In [6]:
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"
pc=Pinecone(api_key=api_key)

In [13]:
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric='dotproduct',##Sparse Values
        spec=ServerlessSpec(cloud="aws",region="us-east-1"),
    )

In [14]:
index=pc.Index(index_name)
index_name

'hybrid-search-langchain-pinecone'

In [19]:
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [20]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder=BM25Encoder().default()
bm25_encoder

In [21]:
sentences=[
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited Tokyo",
]
bm25_encoder.fit(sentences)
bm25_encoder.dump("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 153.29it/s]


In [22]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [23]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x0000028E556EFB50>, index=<pinecone.data.index.Index object at 0x0000028E0878C040>)

In [24]:
retriever.add_texts(
    [
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited Tokyo",
]
)

100%|██████████| 1/1 [00:02<00:00,  2.46s/it]


In [25]:
retriever.invoke("What city did i visit last")

[Document(page_content='In 2022, I visited New York'),
 Document(page_content='In 2021, I visited Tokyo'),
 Document(page_content='In 2023, I visited Paris')]

In [26]:
retriever.invoke("What city did i visit first")

[Document(page_content='In 2022, I visited New York'),
 Document(page_content='In 2021, I visited Tokyo'),
 Document(page_content='In 2023, I visited Paris')]